## 참고: https://hleecaster.com/ml-linear-regression-example/

## 라이브러리 설치, 호출

In [2]:
'''!pip3 install -U scikit-learn
!pip3 install pandas
!pip3 install numpy
!pip3 install matplotlib
!pip3 install statsmodels'''

     ---------------------------------------- 7.3/7.3 MB 4.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.23.2
    Uninstalling scikit-learn-0.23.2:
      Successfully uninstalled scikit-learn-0.23.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pycaret 2.3.10 requires scikit-learn==0.23.2, but you have scikit-learn 1.1.2 which is incompatible.


In [2]:
#!pip install tensorflow

     ------------------------------------ 455.9/455.9 MB 649.1 kB/s eta 0:00:00
     ---------------------------------------- 5.9/5.9 MB 17.9 MB/s eta 0:00:00
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
     ------------------------------------- 895.9/895.9 kB 28.6 MB/s eta 0:00:00
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
     ---------------------------------------- 1.5/1.5 MB 31.4 MB/s eta 0:00:00
  Using cached libclang-14.0.6-py2.py3-none-win_amd64.whl (14.2 MB)
     ---------------------------------------- 2.6/2.6 MB 27.9 MB/s eta 0:00:00
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
     ---------------------------------------- 3.6/3.6 MB 13.7 MB/s eta 0:00:00
  Using cached absl_py-1.2.0-py3-none-any.whl (123 kB)
     ------------------------------------- 438.7/438.7 kB 26.8 MB/s eta 0:00:00
     -----------------

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau

## 데이터 다운로드 (특별할인 판매)

In [2]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/jmnote/zdata/master/logistic-regression/special-sales.csv')
print(df.head())
print(df.shape)

         date weekday  busy_day  high_temperature  special_sales
0  2002-08-05     Mon         0                28              1
1  2002-08-06     Tue         0                24              0
2  2002-08-07     Wed         1                26              0
3  2002-08-08     Thu         0                24              0
4  2002-08-09     Fri         0                23              0
(21, 5)


## Input, Feature 설정

In [3]:
Label = df['special_sales']
InputFeature = df[['busy_day','high_temperature']]

## Keras Logit 모델 fitting

In [4]:
model = Sequential()
model.add(Dense(3, activation='linear', input_shape=(2,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

## Keras 모델 살펴보기

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 3)                 9         
                                                                 
 dense_1 (Dense)             (None, 1)                 4         
                                                                 
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


## Call-back 함수
## 모델 학습

In [6]:
# Call-back 함수
# CheckPoint: Epoch 마다 validation 성능을 검증하여, best performance 일 경우 저장
CP = ModelCheckpoint(filepath='-{epoch:03d}-{loss:.4f}-{accuracy:.4f}.hdf5',
            monitor='loss', verbose=1, save_best_only=True, mode='min')

# Learning Rate 줄여나가기
LR = ReduceLROnPlateau(monitor='loss',factor=0.8,patience=3, verbose=1, min_lr=1e-8)

CALLBACK = [CP, LR]

In [7]:
# model.fit(x=InputFeature, y=Label, epochs=100, shuffle=True, batch_size=3, callbacks=CALLBACK)

# Model Load 하기 전, hdf5 파일 이름 꼭 확인하기

In [8]:
model.load_weights("-098-0.5495-0.7619.hdf5")

## FLASK 셋팅하기

In [9]:
from flask import Flask
from flask import render_template
from flask import request

In [10]:
app = Flask(__name__)

# FLASK API 구현부분

In [11]:
@app.route('/')
@app.route('/SpecialSale')
def PredictionSpecialSale():
    busy_day = request.args.get('busy_day')
    high_temperature = request.args.get('high_temperature')
    
    print(busy_day, high_temperature)
    
    if busy_day == None or high_temperature == None:
        return render_template('SpecialSale.html', Output = '')
    
    Input = pd.DataFrame({
        'busy_day':[int(busy_day)],
        'high_temperature':[float(high_temperature)]
    })
    ModelOutput = model.predict(Input)[0][0]

    return render_template('SpecialSale.html', Output = ModelOutput)

# Flask, port 5000으로 실행

In [12]:
app.run(host='192.168.0.26', port=5554)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://192.168.0.26:5554
Press CTRL+C to quit
192.168.0.26 - - [16/Sep/2022 14:44:10] "GET / HTTP/1.1" 200 -


None None


192.168.0.26 - - [16/Sep/2022 14:44:11] "GET /favicon.ico HTTP/1.1" 404 -


30 29
1/1 [==============================] - 0s 437ms/step


192.168.0.26 - - [16/Sep/2022 14:44:23] "GET /SpecialSale?busy_day=30&high_temperature=29 HTTP/1.1" 200 -


In [ ]:
# http://127.0.0.1:5000/SpecialSale/1/38